In [101]:
import numpy as np # добавляет массивы и математику
import sys 

import time

from matplotlib import pyplot as plt # для графиков

In [102]:
import cv2

In [20]:
last = time.time()


mul = 0
for ai, bi in zip(a, b):
    mul+= ai* bi 

print(mul ,time.time() - last)    

1214590843366 1.8640351295471191


In [21]:
last = time.time()

ma = np.array(a)
mb = np.array(b)

mul = np.dot(ma, mb)
print(mul ,time.time() - last)   

-884901402 1.671391248703003


In [109]:
class MLP:
    def __init__ (self, n_hidden=30, l2=0, epohs=100, eta = 0.001, shuffle=True, minibranch_size=1, seed=42):
        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2 
        self.epohs = epohs
        self.eta = eta
        self.shuffle = shuffle
        self.minebranch_size = minibranch_size
    
    def _onehot (self, y, n_classes):
        onehot = np.zeros( (n_classes, y.shape[0] ) )
        
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1
        
        return onehot.T
    
    def _sigmoid(self, z):
        return 1./(1. + np.exp(-np.clip(z, -250, 250)))
    
    def _forward(self, X):
        # гигасумма для скрытого слоя
        z_h = np.dot(X, self.w_h) + self.b_h
        # активация
        a_h = self._sigmoid(z_h)
        # гигасумма для слоя dsвыходного
        z_out = np.dot(a_h, self.w_out ) + self.b_out
        # активация активация выходного слоя
        a_out = self._sigmoid(z_out)
        
        return z_h, a_h, z_out, a_out
    
    def _compute_cost (self, y_enc, output):
        L2_term = (self.l2 * (np.sum(self.w_h ** 2) + np.sum(self.w_out ** 2) ) )
        
        term1 = -y_enc * (np.log(output))
        term2 = (1 - y_enc) * np.log(1 - output)
        
        cost = np.sum(term1 - term2) + L2_term 
        return cost

In [99]:
mlp = MLP()
mlp2 = MLP()

SyntaxError: invalid syntax (1382138201.py, line 1)

In [90]:
np.clip(a, 1, 3)

array([1, 2, 3])